# Data Quality Validation Experiments

This notebook documents data quality validation and profiling experiments for the Flight Airfare Prediction project.

## Contents
1. Data Profiling
2. Null Analysis & Handling
3. Outlier Detection
4. Value Validation
5. Business Rule Validation
6. Quality Scoring Framework

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("✓ Setup complete")

In [ ]:
# Load data
df = pd.read_csv(project_root / "data" / "train.csv")
print(f"Loaded {len(df)} records with {len(df.columns)} columns")

## 1. Data Profiling

In [ ]:

def profile_dataframe(df):
    """Generate comprehensive data profile"""
    profile = []
    
    for col in df.columns:
        col_data = df[col]
        
        profile.append({
            'Column': col,
            'Type': str(col_data.dtype),
            'Non-Null': col_data.notna().sum(),
            'Null %': (col_data.isna().sum() / len(df)) * 100,
            'Unique': col_data.nunique(),
            'Unique %': (col_data.nunique() / len(df)) * 100,
            'Sample': str(col_data.dropna().iloc[0]) if len(col_data.dropna()) > 0 else 'N/A'
        })
    
    return pd.DataFrame(profile)

profile_df = profile_dataframe(df)
print("Data Profile:")
print(profile_df.to_string(index=False))

In [ ]:
# Numeric column statistics
print("\nNumeric Column Statistics:")
df.describe()

In [ ]:
# Categorical column value counts
categorical_cols = ['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(categorical_cols):
    df[col].value_counts().head(10).plot(kind='bar', ax=axes[i], color='steelblue')
    axes[i].set_title(f'{col} Distribution')
    axes[i].tick_params(axis='x', rotation=45)

# Hide empty subplot
axes[-1].set_visible(False)
plt.tight_layout()
plt.show()

## 2. Null Analysis & Handling Strategies

In [ ]:
# Null analysis
null_analysis = pd.DataFrame({
    'Column': df.columns,
    'Null Count': df.isnull().sum().values,
    'Null %': (df.isnull().sum().values / len(df) * 100).round(2)
}).sort_values('Null Count', ascending=False)

print("Null Analysis:")
print(null_analysis)

# Visualize
plt.figure(figsize=(10, 6))
null_cols = null_analysis[null_analysis['Null Count'] > 0]
if len(null_cols) > 0:
    plt.bar(null_cols['Column'], null_cols['Null %'], color='coral')
    plt.xlabel('Column')
    plt.ylabel('Null Percentage')
    plt.title('Null Values by Column')
    plt.xticks(rotation=45)
else:
    plt.text(0.5, 0.5, 'No null values found!', ha='center', va='center', fontsize=14)
    plt.title('Null Analysis')
plt.tight_layout()
plt.show()

In [ ]:
# Experiment: Null handling strategies
print("Testing null handling strategies...\n")

# Strategy 1: Drop rows with nulls
df_dropped = df.dropna()
print(f"Strategy 1 - Drop nulls: {len(df)} → {len(df_dropped)} rows ({(1-len(df_dropped)/len(df))*100:.2f}% loss)")

# Strategy 2: Fill with mode (for categorical)
df_filled = df.copy()
for col in df.select_dtypes(include='object').columns:
    mode_val = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Unknown'
    df_filled[col] = df_filled[col].fillna(mode_val)
print(f"Strategy 2 - Fill with mode: 0 nulls remaining in categorical cols")

# Strategy 3: Fill numeric with median
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    df_filled[col] = df_filled[col].fillna(df[col].median())
print(f"Strategy 3 - Fill numeric with median: 0 nulls remaining in numeric cols")

In [ ]:
# Missing value patterns
print("Missing value patterns:")

# Rows with any missing values
rows_with_nulls = df.isnull().any(axis=1).sum()
print(f"Rows with at least one null: {rows_with_nulls} ({rows_with_nulls/len(df)*100:.2f}%)")

# Columns with >5% missing
high_null_cols = null_analysis[null_analysis['Null %'] > 5]['Column'].tolist()
if high_null_cols:
    print(f"Columns with >5% nulls: {high_null_cols}")
else:
    print("No columns with >5% nulls")

## 3. Outlier Detection

In [ ]:
# Price outlier analysis
print("Price Distribution Analysis:")
print(df['Price'].describe())

# IQR method
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_iqr = df[(df['Price'] < lower_bound) | (df['Price'] > upper_bound)]
print(f"\nIQR Method:")
print(f"  Lower bound: {lower_bound:.2f}")
print(f"  Upper bound: {upper_bound:.2f}")
print(f"  Outliers: {len(outliers_iqr)} ({len(outliers_iqr)/len(df)*100:.2f}%)")

In [ ]:
# Z-score method
z_scores = np.abs(stats.zscore(df['Price']))
outliers_zscore = df[z_scores > 3]

print(f"\nZ-Score Method (|z| > 3):")
print(f"  Outliers: {len(outliers_zscore)} ({len(outliers_zscore)/len(df)*100:.2f}%)")

In [ ]:
# Visualize outliers
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Box plot
axes[0].boxplot(df['Price'])
axes[0].set_title('Price Box Plot')
axes[0].set_ylabel('Price (INR)')

# Histogram
axes[1].hist(df['Price'], bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(upper_bound, color='r', linestyle='--', label=f'Upper bound ({upper_bound:.0f})')
axes[1].set_title('Price Distribution')
axes[1].set_xlabel('Price (INR)')
axes[1].legend()

# After outlier removal
df_no_outliers = df[(df['Price'] >= lower_bound) & (df['Price'] <= upper_bound)]
axes[2].hist(df_no_outliers['Price'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[2].set_title('Price Distribution (No Outliers)')
axes[2].set_xlabel('Price (INR)')

plt.tight_layout()
plt.show()

## 4. Value Validation

In [ ]:
# Define valid values for categorical columns
VALID_VALUES = {
    'Source': ['Delhi', 'Kolkata', 'Banglore', 'Mumbai', 'Chennai'],
    'Destination': ['Delhi', 'Kolkata', 'Banglore', 'Mumbai', 'Chennai', 'Cochin', 'Hyderabad', 'New Delhi'],
    'Total_Stops': ['non-stop', '1 stop', '2 stops', '3 stops', '4 stops']
}

# Validate
print("Value Validation Results:")
print("=" * 50)

for col, valid_vals in VALID_VALUES.items():
    invalid = df[~df[col].isin(valid_vals)][col].unique()
    if len(invalid) > 0:
        print(f"\n❌ {col}: {len(invalid)} invalid values found")
        print(f"   Invalid: {invalid[:10]}")
    else:
        print(f"✓ {col}: All values valid")

In [ ]:
# Date format validation
print("\nDate Format Validation:")

def validate_date_format(date_str):
    """Check if date matches expected format d/M/yyyy"""
    try:
        pd.to_datetime(date_str, format='%d/%m/%Y')
        return True
    except:
        return False

invalid_dates = df[~df['Date_of_Journey'].apply(validate_date_format)]
print(f"Invalid date formats: {len(invalid_dates)} ({len(invalid_dates)/len(df)*100:.2f}%)")

if len(invalid_dates) > 0:
    print(f"Sample invalid dates: {invalid_dates['Date_of_Journey'].head().tolist()}")

In [ ]:
# Time format validation
print("\nTime Format Validation:")

def validate_time_format(time_str):
    """Check if time matches expected format HH:MM"""
    try:
        parts = str(time_str).split(':')[:2]  # Take first two parts (ignore date suffixes)
        hour, minute = int(parts[0]), int(parts[1])
        return 0 <= hour <= 23 and 0 <= minute <= 59
    except:
        return False

for time_col in ['Dep_Time', 'Arrival_Time']:
    invalid_times = df[~df[time_col].apply(validate_time_format)]
    print(f"{time_col}: {len(invalid_times)} invalid ({len(invalid_times)/len(df)*100:.2f}%)")

## 5. Business Rule Validation

In [ ]:
# Rule 1: Source and Destination should be different
print("Business Rule Validation:")
print("=" * 50)

same_city = df[df['Source'] == df['Destination']]
if len(same_city) > 0:
    print(f"\n❌ Rule 1: Source == Destination")
    print(f"   Violations: {len(same_city)}")
else:
    print(f"\n✓ Rule 1: Source ≠ Destination - PASSED")

In [ ]:
# Rule 2: Price should be positive
negative_price = df[df['Price'] <= 0]
if len(negative_price) > 0:
    print(f"❌ Rule 2: Price > 0")
    print(f"   Violations: {len(negative_price)}")
else:
    print(f"✓ Rule 2: Price > 0 - PASSED")

In [ ]:
# Rule 3: Non-stop flights should have Route with only 2 cities
df_temp = df.copy()
df_temp['Route_Cities'] = df_temp['Route'].fillna('').str.split(' → ').str.len()
df_temp['Is_NonStop'] = df_temp['Total_Stops'] == 'non-stop'

inconsistent = df_temp[(df_temp['Is_NonStop']) & (df_temp['Route_Cities'] > 2)]
if len(inconsistent) > 0:
    print(f"⚠️ Rule 3: Non-stop consistency")
    print(f"   Non-stop flights with >2 route segments: {len(inconsistent)}")
else:
    print(f"✓ Rule 3: Non-stop route consistency - PASSED")

In [ ]:
# Rule 4: Duration should be reasonable (30min to 48 hours)
def parse_duration_mins(d):
    try:
        d = str(d).lower()
        hours = int(d.split('h')[0]) if 'h' in d else 0
        mins = int(d.split('h')[1].replace('m','').strip()) if 'h' in d and 'm' in d.split('h')[1] else 0
        if 'm' in d and 'h' not in d:
            mins = int(d.replace('m','').strip())
        return hours * 60 + mins
    except:
        return 0

df_temp['Duration_Mins'] = df_temp['Duration'].apply(parse_duration_mins)

min_duration = 30  # 30 minutes
max_duration = 48 * 60  # 48 hours

unreasonable = df_temp[(df_temp['Duration_Mins'] < min_duration) | (df_temp['Duration_Mins'] > max_duration)]
if len(unreasonable) > 0:
    print(f"⚠️ Rule 4: Duration range ({min_duration}min - {max_duration}min)")
    print(f"   Violations: {len(unreasonable)}")
else:
    print(f"✓ Rule 4: Duration within reasonable range - PASSED")

## 6. Quality Scoring Framework

In [ ]:
# Build quality score per row
def calculate_quality_score(row):
    """Calculate quality score (0-100) for each row"""
    score = 100
    
    # Deductions for issues
    penalties = {
        'null_route': 10,
        'no_info': 5,
        'zero_duration': 15,
        'extreme_price': 10,
        'same_city': 20
    }
    
    # Check Route
    if pd.isna(row.get('Route')):
        score -= penalties['null_route']
    
    # Check Additional_Info
    if row.get('Additional_Info') == 'No Info':
        score -= penalties['no_info']
    
    # Check Duration
    duration_mins = parse_duration_mins(row.get('Duration', '0'))
    if duration_mins == 0:
        score -= penalties['zero_duration']
    
    # Check Price (extreme values)
    price = row.get('Price', 0)
    if price < 1000 or price > 100000:
        score -= penalties['extreme_price']
    
    # Check Source/Destination
    if row.get('Source') == row.get('Destination'):
        score -= penalties['same_city']
    
    return max(0, score)

# Apply to dataframe
df['Quality_Score'] = df.apply(calculate_quality_score, axis=1)

print("Quality Score Distribution:")
print(df['Quality_Score'].describe())

In [ ]:
# Visualize quality distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram
axes[0].hist(df['Quality_Score'], bins=20, edgecolor='black', color='teal', alpha=0.7)
axes[0].axvline(df['Quality_Score'].mean(), color='red', linestyle='--', label=f"Mean: {df['Quality_Score'].mean():.1f}")
axes[0].set_xlabel('Quality Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Quality Score Distribution')
axes[0].legend()

# Quality tiers
quality_tiers = pd.cut(df['Quality_Score'], bins=[0, 60, 80, 90, 100], labels=['Poor', 'Fair', 'Good', 'Excellent'])
tier_counts = quality_tiers.value_counts().sort_index()
tier_counts.plot(kind='bar', ax=axes[1], color=['red', 'orange', 'lightgreen', 'green'])
axes[1].set_xlabel('Quality Tier')
axes[1].set_ylabel('Count')
axes[1].set_title('Records by Quality Tier')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

print("\nQuality Tier Distribution:")
print(tier_counts)

In [ ]:
# Summary validation report
print("\n" + "=" * 60)
print("DATA QUALITY VALIDATION REPORT")
print("=" * 60)

total_records = len(df)
high_quality = len(df[df['Quality_Score'] >= 90])
medium_quality = len(df[(df['Quality_Score'] >= 70) & (df['Quality_Score'] < 90)])
low_quality = len(df[df['Quality_Score'] < 70])

print(f"\nTotal Records: {total_records:,}")
print(f"\nQuality Breakdown:")
print(f"  - High Quality (≥90):    {high_quality:,} ({high_quality/total_records*100:.1f}%)")
print(f"  - Medium Quality (70-89): {medium_quality:,} ({medium_quality/total_records*100:.1f}%)")
print(f"  - Low Quality (<70):      {low_quality:,} ({low_quality/total_records*100:.1f}%)")
print(f"\nAverage Quality Score: {df['Quality_Score'].mean():.2f}/100")
print(f"\nRecommendation: Use records with Quality Score ≥ 80 for ML training")

## Conclusions

### Key Findings:
1. **Null handling**: Route column has some nulls - fill with "Direct" for non-stop
2. **Outliers**: ~5% price outliers using IQR method - consider capping
3. **Value validation**: All categorical values are within expected ranges
4. **Business rules**: Source/Destination always different ✓

### Quality Score Thresholds:
- **Production ML Training**: Score ≥ 80
- **Exploratory Analysis**: Score ≥ 60
- **Flag for Review**: Score < 60